## Цель работы:
Обучить нейронные сети на корпусе текстов определенного человека, научиться генерировать текст в стиле определенного человека

## Установка требуемых пакетов

In [1]:
'''
# Используется Python 3.11
%pip install ipywidgets python-dotenv ollama tensorboard
# Используем колесо для cuda 12.6 (версия cuda, GPU и драйвера GPU должны быть совместимы между собой)
%pip install --upgrade pip && pip install xformers --index-url https://download.pytorch.org/whl/cu126
# вместе с xformers установился torch, чтобы узнать его версию и сгенерировать правильную команду для установки unsloth можно воспользоваться комнадой:
#!wget -qO- https://raw.githubusercontent.com/unslothai/unsloth/main/unsloth/_auto_install.py | python -
# выполнив предыдущую команду, мы получили команду для установки unsloth:
%pip install "unsloth[cu126-ampere-torch260] @ git+https://github.com/unslothai/unsloth.git"
# flash-attn компилируется крайне долго, устанавливать желательно через консоль
%pip install --no-build-isolation flash-attn
'''

'\n# Используется Python 3.11\n%pip install ipywidgets python-dotenv ollama tensorboard\n# Используем колесо для cuda 12.6 (версия cuda, GPU и драйвера GPU должны быть совместимы между собой)\n%pip install --upgrade pip && pip install xformers --index-url https://download.pytorch.org/whl/cu126\n# вместе с xformers установился torch, чтобы узнать его версию и сгенерировать правильную команду для установки unsloth можно воспользоваться комнадой:\n#!wget -qO- https://raw.githubusercontent.com/unslothai/unsloth/main/unsloth/_auto_install.py | python -\n# выполнив предыдущую команду, мы получили команду для установки unsloth:\n%pip install "unsloth[cu126-ampere-torch260] @ git+https://github.com/unslothai/unsloth.git"\n# flash-attn компилируется крайне долго, устанавливать желательно через консоль\n%pip install --no-build-isolation flash-attn\n'

## Генерация Prompt на основе данных датасета с помощью Ollama

In [2]:
import pandas as pd
import ollama
import os
import subprocess
from datetime import datetime

# Пути к файлам
input_file = "datasets/Lebedev_messages_5436.csv"
output_file = f"{input_file[:-4]}_with_prompt.csv"
ollama_used = False

# Функция для генерации промпта
def generate_prompt(message):
    global ollama_used
    ollama_used = True
    prompt_style = """Вы ведете контент-план блогера, писателя, у которого свой стиль письма. 
    Пожалуйста, составьте краткое задание на русском языке в простой форме для написания следующего текста. Представим, что этого текста еще не существует и блоггер должен его написать.
    Задание не должно содержать форматирование или примеры. 
    
    ### Текст для задания:
    {}"""
    response = ollama.generate(model="deepseek-r1:8b", prompt=prompt_style.format(message))["response"]
    # убираем лишнюю информацию из ответа модели
    response = response.split("</think>")[-1].strip()
    return response

# Загружаем уже обработанные данные, если файл существует
processed_df = pd.read_csv(output_file) if os.path.exists(output_file) else pd.DataFrame()
processed_count = len(processed_df)

# Загружаем исходный датасет и определяем необработанные строки
df = pd.read_csv(input_file)
remaining_df = df[processed_count:]

# Обработка данных чанками
chunk_size = 50
for start in range(0, len(remaining_df), chunk_size):
    chunk = remaining_df[start:start + chunk_size].copy()
    chunk["Prompt"] = chunk["Message"].apply(generate_prompt)

    # Сохраняем результаты (дописываем или создаем файл)
    chunk.to_csv(output_file, mode="a", header=not os.path.exists(output_file), index=False)
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"[{current_time}] Обработано строк: {processed_count + start + len(chunk)}")

print("Генерация завершена и данные сохранены.")
# выгружаем Ollama
if ollama_used:
    subprocess.Popen(["ollama", "stop", "deepseek-r1:8b"])

Генерация завершена и данные сохранены.


In [3]:
import torch
import gc
import time
if not torch.cuda.is_available():
    raise NotImplementedError("NVIDIA GPU not found!") # unsloth работает только с GPU. Дальнейшее выполнее кода не имеет смысла.
from unsloth import FastLanguageModel
max_seq_length = 2048 
dtype = torch.bfloat16 
load_in_4bit = True

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
# Функция для очистки видеопамяти (VRAM)
def clear_vram():
    print("Очистка VRAM...")
    try:
        # Сборка мусора
        gc.collect()
        # Очистка кэша CUDA
        torch.cuda.empty_cache()
        # Сборка межпроцессорной памяти CUDA
        torch.cuda.ipc_collect()
        print("VRAM успешно очищена.")
    except Exception as e:
        print(f"Не удалось очистить VRAM: {e}")

# Функция для вывода статистики VRAM
def display_vram_stats(stage="before", start_gpu_memory=0):
    gpu_stats = torch.cuda.get_device_properties(0)
    current_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
    max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
    if stage == "before":
        print(f"GPU = {gpu_stats.name}.\nИспользуется {current_gpu_memory} GB из {max_memory} GB vRAM.")
        return current_gpu_memory
    elif stage == "after":
        used_memory_for_lora = round(current_gpu_memory - start_gpu_memory, 3)
        used_percentage = round(current_gpu_memory / max_memory * 100, 3)
        lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
        print(f"Обучение длилось {trainer_stats.metrics['train_runtime']} секунд ({round(trainer_stats.metrics['train_runtime']/60, 2)} минут).")
        print(f"Максимально использовалось vRAM = {current_gpu_memory} ГБ.")
        print(f"Максимально использовалось vRAM для обучения = {used_memory_for_lora} ГБ.")
        print(f"Максимально использовалось vRAM в % от общей памяти = {used_percentage} %.")
        print(f"Максимально использовалось vRAM для обучения в % от общей памяти = {lora_percentage} %.")

In [5]:
# Получаем токен Hugging Face и авторизуемся
from huggingface_hub import login
from dotenv import load_dotenv
load_dotenv()
hf_token = os.environ.get('HF_TOKEN')
login(hf_token)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


# DeepSeek R1
## Тестируем модель DeepSeek R1 до обучения

In [6]:
# Загрузка модели
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = hf_token, 
)

F:\anaconda3\envs\unsloth\Lib\site-packages\unsloth_zoo\gradient_checkpointing.py:330: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  GPU_BUFFERS = tuple([torch.empty(2*256*2048, dtype = dtype, device = f"cuda:{i}") for i in range(n_gpus)])


==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.0.
   \\   /|    NVIDIA GeForce RTX 4070 Ti SUPER. Num GPUs = 1. Max memory: 15.992 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.6.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [7]:
prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context. 
Write a response that appropriately completes the request. 
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
Вы являетесь писателем книг и текстовым блоггером живого журнала, со своим уникальным стилем написания текстов. 
Пожалуйста, напишите текст на русском языке по следующему заданию.

### Question:
{}

### Response:
<think>{}"""

In [8]:
question = """Написать текст, критически оценивающий электрочистилки для обуви."""

FastLanguageModel.for_inference(model)  # Unsloth has 2x faster inference!
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("</think>")[1])



Электрочистилки для обуви — это современный и удобный инструмент для тех, кто ценит卫生 и порядок. Однако, как и任何 другой технический устройство, они имеют свои преимущества и недостатки. Вот несколько ключевых аспектов, которые можно рассмотреть при оценке электрочистилок:

### 1. Эффективность
Электрочистилки для обуви используют различные материалы для удаления пыли и загрязнения. Часть моделей оснащены металлическими刷ками, которые эффективно抓гают частицы грязи и шероховатостей. Другие модели могут использовать абразивные наконец, которые помогают удалить глубокие углубления. Выбирайте модели, которые соответствуют вашему типу обуви и частоте использования.

### 2. Безопасность
Электрочистилки подключаются к источнику тока, что делает их использование более эффективным, но и более безопасным. Убедитесь, что устройство соответствует стандартам безопасности, имеет защитные устройства против короткого замыкания и перегрузки. Также проверьте, есть ли автоматическая выключка при достижен

## Обучаем DeepSeek R1

In [9]:
# Настройка модели для обучения с использованием LoRA
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,  
    bias="none",  
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
    random_state=3407,
    use_rslora=False,  
    loftq_config=None,
)

Unsloth 2025.3.19 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [10]:
# Запуск сервиса TensorBoard
subprocess.Popen(["tensorboard", "--logdir", "outputs"])

<Popen: returncode: None args: ['tensorboard', '--logdir', 'outputs']>

In [11]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. 
Write a response that appropriately completes the request. 
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
{}

### Question:
{}

### Response:
<think>

</think>
{}"""
EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN

def formatting_prompts_func(examples):
    instruction = "Вы являетесь писателем книг и текстовым блоггером живого журнала, со своим уникальным стилем написания текстов. Пожалуйста, напишите текст на русском языке по следующему заданию."
    inputs       = examples["Prompt"]
    outputs      = examples["Message"]
    texts = []
    for input, output in zip(inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

In [12]:
from datasets import Dataset
dataset = Dataset.from_pandas(pd.read_csv("datasets/Lebedev_messages_5436_with_prompt.csv"))
formatting_dataset = dataset.map(formatting_prompts_func, batched = True)
formatting_dataset["text"][1]

Map:   0%|          | 0/5436 [00:00<?, ? examples/s]

'Below is an instruction that describes a task, paired with an input that provides further context. \nWrite a response that appropriately completes the request. \nBefore answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.\n\n### Instruction:\nВы являетесь писателем книг и текстовым блоггером живого журнала, со своим уникальным стилем написания текстов. Пожалуйста, напишите текст на русском языке по следующему заданию.\n\n### Question:\nНапишите текст, объясняющий разницу между терминами "загрузка" и "скачивание" в контексте работы с данными на русском языке.\n\n### Response:\n<think>\n\n</think>\nЗагрузка и скачивание\nЛюбопытно, что в русском языке процесс передачи данных наружу (на сервер или в память) называется "загрузка", а процесс получение данных извне - "скачивание".\nПрекрасный пример: "Скачать программы из категории Загрузка".\nТо есть, наружу мы грузим, совершаем действие с чем-то твердым. А к с

In [13]:
# Настройка тренера
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=formatting_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=1,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=8,
        warmup_steps=10,
        max_steps=60,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs/deepseek_r1",
        report_to=["tensorboard"],
    ),
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/5436 [00:00<?, ? examples/s]

In [14]:
# Показываем текущую статистику vRAM
start_gpu_memory = display_vram_stats()

GPU = NVIDIA GeForce RTX 4070 Ti SUPER.
Используется 6.051 GB из 15.992 GB vRAM.


In [15]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 5,436 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,2.802000
20,1.852000
30,1.646300
40,1.611100
50,1.526200
60,1.487200


In [16]:
# Показываем окончательную статистику памяти и времени
display_vram_stats("after", start_gpu_memory)

Обучение длилось 357.3391 секунд (5.96 минут).
Максимально использовалось vRAM = 8.852 ГБ.
Максимально использовалось vRAM для обучения = 2.801 ГБ.
Максимально использовалось vRAM в % от общей памяти = 55.353 %.
Максимально использовалось vRAM для обучения в % от общей памяти = 17.515 %.


## Тестируем модель DeepSeek R1 после обучения

In [17]:
question = "Написать текст, критически оценивающий электрочистилки для обуви."

FastLanguageModel.for_inference(model)  # Unsloth has 2x faster inference!
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("</think>")[1])


Электрочистилки для обуви
Электрочистилки для обуви - это хуйня. 
Если у вас есть электрочистилка для обуви, то вы себе пиздец не чистите обуви. 
Вот что будет: вы включаете электрочистилку для обуви, она запускается, вы кладете обуви на нее, она нахуй не чистит. 
А это потому, что электрочистилка для обуви не может ничего сделать с обуви. 
А если вы хотите, чтобы обуя была чистой, то вы просто сначала ее вымочите.<｜end▁of▁sentence｜>


## Сохраняем адаптеры LoRA и модель

In [18]:
model.save_pretrained("Lebedev_DeepSeek-R1")
tokenizer.save_pretrained("Lebedev_DeepSeek-R1")
model.save_pretrained_merged("Lebedev_DeepSeek-R1-finetune", tokenizer)
del model
del tokenizer
del inputs
del outputs
del trainer
del response
del trainer_stats
del formatting_dataset
clear_vram()

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 75.13 out of 127.91 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 41%|████      | 13/32 [00:00<00:00, 20.92it/s]
We will save to Disk and not RAM now.
100%|██████████| 32/32 [00:13<00:00,  2.34it/s]


Unsloth: Saving tokenizer... Done.
Done.
Очистка VRAM...
VRAM успешно очищена.


# Gemma 2
## Тестируем модель Gemma 2 до обучения

In [19]:
# Загрузка модели
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/gemma-2-9b-it-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

==((====))==  Unsloth 2025.3.19: Fast Gemma2 patching. Transformers: 4.50.0.
   \\   /|    NVIDIA GeForce RTX 4070 Ti SUPER. Num GPUs = 1. Max memory: 15.992 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.6.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [20]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Вы являетесь писателем книг и текстовым блоггером живого журнала, со своим уникальным стилем написания текстов. Пожалуйста, напишите текст на русском языке по следующему заданию.", # instruction
        "Написать короткий текст, объясняющий, почему важно соблюдать правила личной гигиены в общественных местах.", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=500,
    temperature = 1.0, top_p = 0.95, top_k = 64,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])


 
В метро, в автобусе, в очереди за кофе – мы все время где-то рядом друг с другом. И это здорово, ведь общение – основа жизни! Но есть одно "но", которое не так приятно – несоблюдение личной гигиены. 

Представьте: вы едете в переполненном автобусе, а рядом с вами человек, который не мылся уже несколько дней. Или вы стоите в очереди, и кто-то рядом с вами громко чихает, не прикрывая рот. Неприятно, правда? 

Важно помнить, что личная гигиена – это не только про нас самих, но и про тех, кто рядом. Мы все хотим чувствовать себя комфортно и безопасно в общественных местах. А это возможно только тогда, когда каждый из нас заботится о своей чистоте и порядке. 

Так что давайте быть вежливыми друг к другу и следить за своей гигиеной! Ведь это несложно, а польза от этого огромная. 


<end_of_turn>
<eos>


## Обучаем Gemma 2

In [21]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

# Подготовка данных для обучения
EOS_TOKEN = tokenizer.eos_token  # Добавляем EOS-токен (используется в formatting_prompts_func)

formatting_dataset = dataset.map(formatting_prompts_func, batched=True)
formatting_dataset[1]

Unsloth 2025.3.19 patched 42 layers with 42 QKV layers, 42 O layers and 42 MLP layers.


Map:   0%|          | 0/5436 [00:00<?, ? examples/s]

{'Message': 'Загрузка и скачивание\nЛюбопытно, что в русском языке процесс передачи данных наружу (на сервер или в память) называется "загрузка", а процесс получение данных извне - "скачивание".\nПрекрасный пример: "Скачать программы из категории Загрузка".\nТо есть, наружу мы грузим, совершаем действие с чем-то твердым. А к себе мы скачиваем, то есть, совершаем действие с чем-то жидким и бесформенным.',
 'Prompt': 'Напишите текст, объясняющий разницу между терминами "загрузка" и "скачивание" в контексте работы с данными на русском языке.',
 'text': 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nВы являетесь писателем книг и текстовым блоггером живого журнала, со своим уникальным стилем написания текстов. Пожалуйста, напишите текст на русском языке по следующему заданию.\n\n### Input:\nНапишите текст, объясняющий разницу между терминами "загрузка" и "скач

In [22]:
# Настройка тренера для Gemma2
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=formatting_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 1,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir="outputs/gemma2",
        report_to=["tensorboard"],
    ),
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/5436 [00:00<?, ? examples/s]

In [23]:
# Показываем текущую статистику vRAM
start_gpu_memory = display_vram_stats()

GPU = NVIDIA GeForce RTX 4070 Ti SUPER.
Используется 13.717 GB из 15.992 GB vRAM.


In [24]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 5,436 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 54,018,048/9,000,000,000 (0.60% trained)


Step,Training Loss
10,2.333700
20,1.683800
30,1.665800
40,1.589600
50,1.573700
60,1.644000


In [25]:
# Показываем окончательную статистику памяти и времени
display_vram_stats("after", start_gpu_memory)

Обучение длилось 192.2821 секунд (3.2 минут).
Максимально использовалось vRAM = 13.717 ГБ.
Максимально использовалось vRAM для обучения = 0.0 ГБ.
Максимально использовалось vRAM в % от общей памяти = 85.774 %.
Максимально использовалось vRAM для обучения в % от общей памяти = 0.0 %.


## Тестируем модель Gemma 2 после обучения

In [26]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Вы являетесь писателем книг и текстовым блоггером живого журнала, со своим уникальным стилем написания текстов. Пожалуйста, напишите текст на русском языке по следующему заданию.", # instruction
        "Написать текст, объясняющий, почему важно соблюдать правила личной гигиены в общественных местах.", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=500,
    temperature = 1.0, top_p = 0.95, top_k = 64,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])


Личная гигиена
В общественных местах важно соблюдать личную гигиену. Это касается не только мытья рук, но и других аспектов. Например, не стоит есть в общественном транспорте, так как это может привести к попаданию крошек на сиденье и в воздух. Также важно не разговаривать по телефону в общественном транспорте, так как это может быть неприятно для других пассажиров.
Важно помнить, что личная гигиена - это не только забота о себе, но и забота о других людях.<eos>


## Сохраняем адаптеры LoRA и модель

In [27]:
model.save_pretrained("Lebedev_Gemma2")
tokenizer.save_pretrained("Lebedev_Gemma2")
model.save_pretrained_merged("Lebedev_Gemma2-finetune", tokenizer)
del model
del tokenizer
del inputs
del outputs
del trainer
del response
del trainer_stats
del formatting_dataset
clear_vram()

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 74.77 out of 127.91 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 42/42 [00:21<00:00,  1.95it/s]


Unsloth: Saving tokenizer... Done.
Done.
Очистка VRAM...
VRAM успешно очищена.


# Llama 3.1
## Тестируем модель Llama 3.1 до обучения

In [28]:
# Загрузка модели
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B-Instruct-unsloth-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.0.
   \\   /|    NVIDIA GeForce RTX 4070 Ti SUPER. Num GPUs = 1. Max memory: 15.992 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.6.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [29]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Вы являетесь писателем книг и текстовым блоггером живого журнала, со своим уникальным стилем написания текстов. Пожалуйста, напишите текст на русском языке по следующему заданию.", # instruction
        "Написать короткий текст, объясняющий, почему важно соблюдать правила личной гигиены в общественных местах.", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=500,
    temperature = 1.5, min_p = 0.1,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])


Дневник.

Здоровье не только физическое, но и гигиеническое. Чтобы защитить своих же себя, а также окружающих людей от распространения инфекций, важно соблюдать простейшие нормы личной гигиены. Делайте это просто. Прежде всего в местах с большой общественной активностью - общественные туалеты в школах, торговых центрах или на транспорте.
Делайте это в таких местах. 
Например в общественных местах: на массовых событиях. Следите за собой во время физических нагрузок и после. После того, как Вы были в местах общественной активности. Это очень просто. Умный способ сдерживания распространения инфекции, и в данном случае это ваш долг. Всегда помните о важности личной гигиены. Применяйте принцип "человек-окружающая среда" - Вы все-таки находитеесь в ней. 

### Внимание:
Если в Вашей стране существует законная норма личной гигиены, описанная в тексте не стоит ориентироваться на них при написании текста. Описание норм личной гигиены может включать в себя элементы гигиеники и предупреждения отн

## Обучаем Llama 3.1

In [30]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

# Подготовка данных для обучения
EOS_TOKEN = tokenizer.eos_token  # Добавляем EOS-токен (используется в formatting_prompts_func)

formatting_dataset = dataset.map(formatting_prompts_func, batched=True)
formatting_dataset[1]

Map:   0%|          | 0/5436 [00:00<?, ? examples/s]

{'Message': 'Загрузка и скачивание\nЛюбопытно, что в русском языке процесс передачи данных наружу (на сервер или в память) называется "загрузка", а процесс получение данных извне - "скачивание".\nПрекрасный пример: "Скачать программы из категории Загрузка".\nТо есть, наружу мы грузим, совершаем действие с чем-то твердым. А к себе мы скачиваем, то есть, совершаем действие с чем-то жидким и бесформенным.',
 'Prompt': 'Напишите текст, объясняющий разницу между терминами "загрузка" и "скачивание" в контексте работы с данными на русском языке.',
 'text': 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nВы являетесь писателем книг и текстовым блоггером живого журнала, со своим уникальным стилем написания текстов. Пожалуйста, напишите текст на русском языке по следующему заданию.\n\n### Input:\nНапишите текст, объясняющий разницу между терминами "загрузка" и "скач

In [31]:
# Настройка тренера для Llama 3.1
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=formatting_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 1,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir="outputs/llama31",
        report_to=["tensorboard"],
    ),
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/5436 [00:00<?, ? examples/s]

In [32]:
# Показываем текущую статистику vRAM
start_gpu_memory = display_vram_stats()

GPU = NVIDIA GeForce RTX 4070 Ti SUPER.
Используется 15.547 GB из 15.992 GB vRAM.


In [33]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 5,436 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Step,Training Loss
10,2.093800
20,1.539000
30,1.541900
40,1.468500
50,1.456000
60,1.519700


In [34]:
# Показываем окончательную статистику памяти и времени
display_vram_stats("after", start_gpu_memory)

Обучение длилось 170.8137 секунд (2.85 минут).
Максимально использовалось vRAM = 15.547 ГБ.
Максимально использовалось vRAM для обучения = 0.0 ГБ.
Максимально использовалось vRAM в % от общей памяти = 97.217 %.
Максимально использовалось vRAM для обучения в % от общей памяти = 0.0 %.


## Тестируем модель Llama 3.1 после обучения

In [35]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Вы являетесь писателем книг и текстовым блоггером живого журнала, со своим уникальным стилем написания текстов. Пожалуйста, напишите текст на русском языке по следующему заданию.", # instruction
        "Написать текст, объясняющий, почему важно соблюдать правила личной гигиены в общественных местах.", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=500,
    temperature = 1.5, min_p = 0.1,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])



Личная гигиена в общественных местах
Очень важный вопрос. Считается, что личная гигиена - это все, что происходит в душе и на банных катафалках, в санузле, душевой и на помойке, а в остальных частях общественных мест нужно обращаться как можно меньше. 
В частности, при посещении общественных туалетов нужно только опустить нахуй, ни при чем в дальнейшем в туалете не подлизаться, не пиздить, не потрогать себя, не вздохнуть в сторону сиска.
Во всем остальном не важно. В бассейнах можно есть, курить, пить, жрать, драть и подлизываться как угодно. Но только на помойке можно не пиздить. И только на помойке нужно вообще не подлизываться. А на всех остальных местах в бассейнах - как хочешь.
И не важно, чем ты занялся в санузле или душе или помойке, все будет полезно и полезнее, чем если бы ты подлизывался или дрочил или пиздил или вздохнул в сторону сиска. 
Где вы находитесь в общественных местах, личная гигиена никогда не сможет быть чем-то дурным. 
Большинство людей все равно никаких личных

## Сохраняем адаптеры LoRA и модель

In [36]:
model.save_pretrained("Lebedev_Llama31")
tokenizer.save_pretrained("Lebedev_Llama31")
model.save_pretrained_merged("Lebedev_Llama31-finetune", tokenizer)
del model
del tokenizer
del inputs
del outputs
del trainer
del response
del trainer_stats
del formatting_dataset
clear_vram()

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 74.86 out of 127.91 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 32/32 [00:13<00:00,  2.41it/s]


Unsloth: Saving tokenizer... Done.
Done.
Очистка VRAM...
VRAM успешно очищена.


# Сравнение и вывод

### Сравнение моделей

1. DeepSeek R1:

   • До обучения: Модель демонстрировала способность генерировать текст на русском языке в заданном стиле, но в некоторых случаях использовала формальный и структурированный стиль.

   • После обучения: Модель смогла адаптироваться к стилю блоггера, включая использование некорректной лексики. Однако в некоторых случаях текст был чрезмерно грубым и выходил за рамки приемлемого тона.

2. Gemma 2:

   • До обучения: Модель генерировала тексты в более нейтральной и формальной манере, что не соответствовало заданному стилю.

   • После обучения: Модель улучшила стиль, но сохраняла некоторую нейтральность. Некорректная лексика использовалась реже, чем в DeepSeek R1.

3. Llama 3.1:

   • До обучения: Генерация текста была формальной, с некоторыми отклонениями в сторону избыточной информации.

   • После обучения: Модель адаптировалась к заданному стилю, но иногда текст становился слишком грубым и неструктурированным. В некоторых случаях модель генерировала неприемлемые высказывания.

---

### Итоговый вывод

• Лучшая модель: DeepSeek R1. Она лучше всего адаптировалась к заданному стилю (простой язык, некорректная лексика). Несмотря на некоторые грубые ошибки, модель демонстрировала наиболее близкий к заданному стиль текста.

• Gemma 2 также показала хорошие результаты, но ее тексты были более формальными, хоть и в большей степени осмысленными.

• Llama 3.1 оказалась самой производительной, но тексты были слишком грубыми, не связанными, и не всегда соответствовали стилю.

Для задачи генерации текста в стиле блоггера больше подходит DeepSeek R1, так как она лучше всего соответствовала стилю и задаче. Однако, если требуется более мягкий стиль с минимальными отклонениями, можно рассмотреть Gemma 2.